In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../codes/")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../codes/scpy2/")))

from scpy2.utils.nbmagics import install_magics
install_magics()
del install_magics

D:\Program_File\VSCode_Portable\workon_env\pyElan\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
d:\09_Work\Jun_Workspace\Python_Scientific_Computing_Best_Practices\繁中程式碼\codes\scpy2\utils\program_finder.py:25: UserWarning: Inkscape not found
  warnings.warn("Inkscape not found")


In [2]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

plt.rcParams["font.family"] = "simsun" # 可以直接修改設定字典，設定預設字型，這樣就不需要在每次繪製文字時設定字型了
plt.rcParams['axes.unicode_minus']=False    # 當座標軸上的 負號無法正常顯示時，需加這行 code

In [3]:
import numpy as np
import math

## 實用技巧

本節介紹一些透過 numpy 陣列共用記憶體的方法。

### 動態陣列

numpy 的陣列物件不能像清單一樣動態地改變其大小，在做資料獲取的時候，需要頻繁地往陣列中增加資料時很不方便。而 python 標準函數庫中的 `array` 陣列提供了動態分配記憶體的功能，而且它和 numpy 陣列一樣直接將數值的二進位資料儲存在一塊記憶體中，因此我們可以先用 `array` 陣列收集資料，然後透過 `np.frombuffer()` 將 `array` 陣列的資料記憶體直接轉為 numpy 陣列。下面是一個實例：

In [5]:
import numpy as np
from array import array
a = array("d", [1,2,3,4])   # 建立一個array陣列
# 透過np.frombuffer()建立一個和a共享記憶體的NumPy陣列
na = np.frombuffer(a, dtype=np.float32) 
print(a)
print(na)
na[1] = 20  # 修改NumPy陣列中的第一個元素
print(a)

array('d', [1.0, 2.0, 3.0, 4.0])
[0.    1.875 0.    2.    0.    2.125 0.    2.25 ]
array('d', [134217728.0, 2.0, 3.0, 4.0])


`array` 陣列只支援一維，如果我們要擷取多個通導的資料，可以將這些資料依次增加進 `array` 陣列，然後透過 `reshape()` 方法將 `np.frombuffer()` 所建立的 numpy 陣列改為二維陣列。  
在下面的實例中，我們透過 `array` 陣列 `buf` 擷取兩個通道的資料，資料獲取完畢之後，透過 `np.frombuffer()` 將其轉為 numpy 陣列，並透過 `reshape()` 將其形狀改為二維陣列：

In [6]:
import math
buf = array("d")
for i in range(5):
    buf.append(math.sin(i*0.1)) 
    buf.append(math.cos(i*0.1))

data = np.frombuffer(buf, dtype=np.float).reshape(-1, 2)
print( data )

[[0.         1.        ]
 [0.09983342 0.99500417]
 [0.19866933 0.98006658]
 [0.29552021 0.95533649]
 [0.38941834 0.92106099]]


<ipython-input-6-a3d3f818e662>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.frombuffer(buf, dtype=np.float).reshape(-1, 2)


下面是 python 中實現 `array` 物件動態增加元素的演算法：
- `array` 物件擁有一塊用於儲存資料的記憶體，其長度通常比陣列中的所有資料的位元組數要長。
- 當往 `array` 中增加資料時，如果資料記憶體中還有空餘位置，則直接寫入空格位置。
- 當資料記憶體中無空餘位置時，則重新分配一塊更大的資料記憶體，並將目前的資料都複製到這塊新的記憶體中，而舊的資料記憶體則被釋放掉。

根據上述演算法可知，只要往 `array` 中增加元素，其資料記憶體的位址就可能發生改變。在此之前透過 `np.frombuffer()` 建立的陣列仍然參考舊的資料記憶體，進一步成為「野指標」。下面的程式示範了這個過程。其中 `array.buffer_info()` 獲得資料記憶體的位址以及其中有效資料的個數。

In [8]:
a = array("d")
for i in range(10):
    a.append(i)
    if i == 2:
        na = np.frombuffer(a, dtype=float)
    print( a.buffer_info(), end='')
    if i == 4:
        print()

(2662406405552, 1)(2662406405552, 2)(2662406405552, 3)(2662406405552, 4)(2662394828336, 5)
(2662394828336, 6)(2662394828336, 7)(2662394828336, 8)(2662406482992, 9)(2662406482992, 10)

由上面的結果可知，當陣列 a 的長度為 5 和 9 時，資料記憶體被重新分配了。而 na 陣列是在 a 的長度為 3 時透過 `np.frombuffer()` 獲得的，因此它的資料指標己經成為野指標。`ndarray.ctypes.data` 可以獲得陣列的資料記憶體的位，可以看出 na 的資料記憶體位址仍然是 a 在重新分配之前的位，而 na 中的資料也變成了隨機的無效資料。

In [9]:
print( na.ctypes.data )
print( na )

2662406405552
[1.31540359e-311 1.31523495e-311 4.94065646e-324]


由上面的分析可知，每次動態陣列的長度改變時，我們都需要重新呼叫 `np.frombuffer()` 以建立一個新的 `ndarray` 陣列物件來存取其中的資料。  

當每個通道的資料類型不同時，就不能採用 `array.array` 物件了。這時可以使用 `bytearray` 收集資料。`bytearray` 是位元組陣列，因此首先需要透過 `struct` 模組將 python 的數值轉換成其位元組表示形式。如果資料來自二進位檔案或硬體，那麼很可能獲得的己經是位元組資料了，這個步驟可以省略。下面是使用 `bytearray` 進行資料獲取的實例：

> **TIP**

> `bytearray`物件的`+=`運算與其`extend()`方法的功能相同，但`+=`的執行速度要比`extend()`快許多，讀者可以使用`%timeit`自行驗證。

In [4]:
import struct
buf = bytearray()
for i in range(5):
    buf += struct.pack("=hdd", i, math.sin(i*0.1), math.cos(i*0.1)) #❶

dtype = np.dtype({"names":["id","sin","cos"], "formats":["h", "d", "d"]}) #❷
data = np.frombuffer(buf, dtype=dtype) #❸
print( data )

[(0, 0.0, 1.0) (1, 0.09983341664682815, 0.9950041652780258)
 (2, 0.19866933079506122, 0.9800665778412416)
 (3, 0.2955202066613396, 0.955336489125606)
 (4, 0.3894183423086505, 0.9210609940028851)]


❶ 擷取三個通道的資料，其中通道 1 是短整數，其類型符號為 "h" ，通道 2 和 3 為雙精度浮點數，其類型符號為 "d"。類型格式字串中的 "=" 表示輸出的位元組資料不進行記憶體對齊。即一筆資料的位元組數為 2+8+8=18，如果沒有"="，那麼一筆資料的位元組數為 8+8+8=24。  
❷ 定義一個 `dtype` 物件來表示一筆資料的結構，`dtype`物件預設不進行記憶體對齊。如果擷取資料用的 `bytearray` 中的資料是記憶體對齊的話，只需要設定 `dtype()` 的 `align` 參數為 True 即可。  
❸ 最後透過 `np.frombuffer()` 將 `bytearray` 轉為 numpy 的結構陣列。然後就可以透過 `data["id"]`, `data["sin"]`, `data["cos"]` 存取這三個通道的資料了。

### 和其它物件共享記憶體

當其他物件提供了取得其內部資料存取區的介面時，可以是用 `np.frombuffer()` 建立一個陣列與此物件共用資料記憶體。如果物件沒有提供介面，但是能夠取得資料儲存區的位址，可以透過 `ctypes` 和 `numpy.ctypeslib` 模組中提供的函數，建立與物件共用記憶體的陣列。下面以 PyQt4 中的 QImage 物件為例，介紹如何建立一個與 QImage 物件共用記憶體的陣列。

首先建立一個 QImage 物件，並載入 "lena.png" 檔案中的內容。然後輸出與影像相關的一些資訊，為了建立與該影像共用記憶體的陣列，我們需要使用這些資訊。

In [18]:
from PyQt5.QtGui import QImage, qRgb
img = QImage("lena.png")
print( "width & height:", img.width(), img.height() )
print( "depth:", img.depth() ) #每個像素的位元數
print( "format:", img.format(), QImage.Format_RGB32 )
print( "byteCount:", img.byteCount() ) #圖形的總位元組數
print( "bytesPerLine:", img.bytesPerLine() ) #每行的位元組數
print( "bits:", int(img.bits()) ) #圖形第一個位元組的位址

width & height: 512 393
depth: 32
format: 4 4
byteCount: 804864
bytesPerLine: 2048
bits: 2662375624896


❶ 由於我們只知道資料的位址，首先需要使用 `ctypes.cast()` 將整數轉為一個指向單字節類型的指標。  
❷ 然後使用 `np.ctypeslib.as_array()` 將 `ctypes` 的指標指向的記憶體轉換成 numpy 陣列。`as_array()` 的第二個參數是該陣列的形狀，注意陣列的第 0 軸為影像的高，第 1 軸為影像的寬，第 2 軸為每個像素的位元組數。

In [19]:
import ctypes
addr = int(img.bits())
pointer = ctypes.cast(addr, ctypes.POINTER(ctypes.c_uint8)) #❶
arr = np.ctypeslib.as_array(pointer, (img.height(), img.width(), img.depth()//8)) #❷

下面透過  arr 陣列和 img 物件檢視位於像素座標 (50, 100) 處的像素顏色值，可以看到二者是完全相同的：

In [20]:
x, y = 100, 50
b, g, r, a = arr[y, x]
print( qRgb(r, g, b) )
print( img.pixel(x, y) )

4289282380
4289282380


下面透過 arr 陣列修改顏色值，並透過 img 物件檢視修改的結果，由結果可知二者的確共用著同一塊記憶體：

In [21]:
arr[y, x, :3] = 0x12, 0x34, 0x56
print( hex(img.pixel(x, y)) )

0xff563412


使用上述方法共用記憶體時需注意必須保持目標物件處於可存取狀態。例如在上例中，如果執行 `del img` 敘述引起 img 物件被垃圾回收，則透過 arr 陣列將存取被釋放掉的記憶體區域，為了解決這個問題，可以讓陣列的 `base` 屬性參考目標物件，這樣只要陣列不被釋放，則目標物件也不會被釋放。為了能正確設定 `base` 屬性，需要使用陣列的 `__array_interface__` 介面。

❶ 在呼叫 `array()` 將目標物件轉換成陣列時，如果目標物件擁有 `__array_interface__` 屬性，則根據該屬性的描述建立陣列。它是一個具有特定鍵值的字典，參見表2-10。

表2-10 鍵值及含義
| 鍵值 | 含義 |
|-----|------|
| shape | 所建立陣列的形狀 |
| data | 資料儲存區的啟始位，以及是否唯讀 |
| strides | 陣列的 strides 屬性 |
| typestr | 元素類型描述符號 |
| descr | 如果建立結構陣列，該鍵描述結構各個欄位名稱以及對應的資料類型 |
| version | 固定為 3 |

❷ 設定 `copy` 參數為 False，這樣所建立的陣列與目標物件共用記憶體，否則將複製目標物件的記憶體。  
❸ 在建立完陣列之後，可以刪除 `__array_interface__` 屬性。  
❹ 所得到的陣列 arr2 與 arr 相同，並且其 `base` 屬性為 img 物件。

In [22]:
interface = {
    'shape': (img.height(), img.width(), 4),
    'data': (int(img.bits()), False),
    'strides': (img.bytesPerLine(), 4, 1),
    'typestr': "|u1",
    'version': 3,
}

img.__array_interface__ = interface #❶

arr2 = np.array(img, copy=False)  #❷
del img.__array_interface__ #❸
print( np.all(arr2 == arr), arr2.base is img )  #❹ 

True True


如果目標物件唯讀，無法為其增加 `__array_interface__` 屬性，可以建立一個代理用的 ArrayProxy 物件，在該代理物件中參考目標物件，使其不會被垃圾回收，同時提供 `__array_interface__` 屬性，以供建立對應的陣列。

In [23]:
class ArrayProxy(object):
    def __init__(self, base, interface):
        self.base = base
        self.__array_interface__ = interface
        
arr3 = np.array(ArrayProxy(img, interface), copy=False)
print( np.all(arr3 == arr) )

True


### 與結構陣列共享記憶體

從結構陣列取得某個欄位時，獲得的是原陣列的視圖，但是如果取得多個欄位，將獲得一個全新的陣列，不與原陣列共用記憶體。

In [24]:
persontype = np.dtype({
    'names':['name', 'age', 'weight', 'height'],
    'formats':['S30','i', 'f', 'f']}, align= True )
a = np.array([("Zhang", 32, 72.5, 167.0), 
              ("Wang", 24, 65.2, 170.0)], dtype=persontype)

print( a["age"].base is a ) #檢視
print( a[["age", "height"]].base is None ) #複製

True
False


為了建立結陣列的多欄位視圖，可以使用下面的 fields_view() 函數。它透過原陣列的 `dtype` 屬性建立視圖陣列的 `dtype` 物件。然後透過 `ndarray()` 建立視圖陣列。

In [25]:
def fields_view(arr, fields):
    dtype2 = np.dtype({name:arr.dtype.fields[name] for name in fields})
    return np.ndarray(arr.shape, dtype2, arr, 0, arr.strides)

v = fields_view(a, ["age", "weight"])
print( v.base is a )

v["age"] += 10
print( a )

True
[(b'Zhang', 42, 72.5, 167.) (b'Wang', 34, 65.2, 170.)]


`dtype` 物件的 `fields` 屬性是一個以欄位名為鍵、以欄位類型和位元組偏移量為值的字典，使用它建立新的 `dtype` 物件時，可以保持欄位的偏移量：

In [26]:
print( a.dtype.fields )
print( a.dtype )
print( v.dtype )

{'name': (dtype('S30'), 0), 'age': (dtype('int32'), 32), 'weight': (dtype('float32'), 36), 'height': (dtype('float32'), 40)}
{'names': ['name', 'age', 'weight', 'height'], 'formats': ['S30', '<i4', '<f4', '<f4'], 'offsets': [0, 32, 36, 40], 'itemsize': 44, 'aligned': True}
{'names': ['age', 'weight'], 'formats': ['<i4', '<f4'], 'offsets': [32, 36], 'itemsize': 40}


如果這兩個 `dtype` 物件的 `itemsize` 屬性相同，那麼可以使用陣列的 `view()` 方法建立視圖物件。但是從上面的輸出可以看到兩個 `dtype` 物件的位元組數並不相同，一個是 44 個位元組，另一個是 40 個位元組。遇到這種情況時，可以使用 `ndarray()` 建立陣列的視圖，它的呼叫參數如下：
```python
ndarray(shape, dtype=float, buffer=None, offset=0, strides=None, order=None)
```
- shape: 所建立陣列的形狀。
- dtype: 陣列元素類型的 `dtype` 物件。
- buffer: 擁有 buffer 介面的物件，所建立的陣列將與該物件共用記憶體。
- offset: buffer 物件的資料記憶體中的起始位址的偏移量。
- strides: 所建立陣列的 strides 屬性，即每個軸上的索引增加 1 時的位址增量。
- order: C 語言格式或 Fortran 語言格式。

在 field_view() 中，我們所建立的陣列視圖與原陣列擁有相同的 `shape`, `data`, `strides` 屬性。而 `dtype` 屬性中的欄位與原陣列擁有相同的偏移量，顯然這樣的新陣列能夠與原陣列共用記憶體。